In [1]:
import pandas as pd
import sqlite3 as db
import json
from pymystem3 import Mystem
from gensim.models import KeyedVectors as Word2Vec
from tqdm import tqdm

In [2]:
mydb = db.connect("db2.sqlite")
mystem = Mystem()

In [ ]:
#Занесем наш датасет в БД
for df in  pd.read_csv("1.csv",sep = "\t\t", header=None, skiprows=0, chunksize=100000):
    i += 1
    print(i)
    df.to_sql("DATA", mydb, index=False, flavor="sqlite", if_exists='append')

In [ ]:
def tokenize(s):
    res = []
    for x in mystem.analyze(s):
        if ('analysis' in x) and len(x['analysis']) > 0:
            txt = x['analysis'][0]['lex']
            res.append(txt)
    return res

In [ ]:
#считаем
sql = '''select "0", "1" from data'''

cur = mydb.execute(sql)
tst = cur.fetchall()

In [ ]:
#построим обратный индекс
sql = """CREATE TABLE rindex(id int PRIMIRY KEY, word Text)"""
mydb.execute(sql) 
mydb.commit()

sql = lambda a,b:'''INSERT INTO rindex(word, qid) VALUES ("{0}", "{1}")'''.format(a,b)
i = 0
for x in tqdm(tst):
    if x[1] != None:
        for y in tokenize(x[1]):
            mydb.execute(sql(y, x[0])) 
    i += 1
    if i%300000 == 0:
        mydb.commit()

In [ ]:
#сджоиним обратный индекс и датасет чтобы ускорить поиск запросов по вхождению
mydb.execute("""CREATE TABLE myjoin(id int PRIMIRY KEY, word Text,  q Text, a Text)""") 
mydb.execute('''Insert into myjoin select rindex2.id as id, rindex2.word as word, data."1" as q, data."2" as a
                  from rindex2 inner join data on rindex2.qid = data."0"''')
mydb.execute('''Create index myjoin_idx on myjoin(word, q, a)''')
mydb.commit()